In [7]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

driver = webdriver.Chrome()

## Linkedin scraper

### Signing in

In [9]:
driver.get('https://www.linkedin.com')

# getting username and pwd fields
username_field = driver.find_element(By.ID, "session_key")
password_field = driver.find_element(By.ID, "session_password")
# submit_button  = driver.find_element(by='css selector', value="input[type='submit']")
submit_button  = driver.find_element(By.CSS_SELECTOR, "[type='submit']")

# importing username and pwd from credentials.json
with open('./info.json', 'r') as f:
    info = json.load(f)

username_field.send_keys(info['username'])
password_field.send_keys(info['password'])

submit_button.click()

### Job page

In [30]:
driver.get('https://www.linkedin.com/jobs/?=')

interested_roles = info['interested_roles']

job_dict = {
            "Title": [], 
            "Company": [],
            "Location": [],
            "Description": [], 
            "URL": []
           }

for i, interested_role in enumerate(interested_roles):
    search_field = driver.find_element(By.CSS_SELECTOR, "[id^='jobs-search-box-keyword-id'")
    search_field.clear()
    search_field.send_keys(interested_role, Keys.ENTER)
    
    time.sleep(5)
    
    # Filter by "experience level"
    # have to do this only once
    if i == 0:
        job_filter = driver.find_element(By.CLASS_NAME, "search-reusables__filter-list")
        exp_filter = job_filter.find_elements(By.CLASS_NAME, "search-reusables__primary-filter")[2]
        exp_filter.click()

        entry_lvl_filter = driver.find_element(By.ID, "experience-2")
        driver.execute_script("arguments[0].click();", entry_lvl_filter)
        driver.find_elements(By.CLASS_NAME, "artdeco-button--primary")[1].click()

        time.sleep(2)
    
    job_list = driver.find_element(By.CLASS_NAME, "scaffold-layout__list-container")
    print(job_list)
    
    job_links = job_list.find_elements(By.CLASS_NAME, "jobs-search-results__list-item")
    
    for job_link in job_links[:3]:
        job_link.click()
        
        job_details = driver.find_element(By.ID, "job-details")
        job_title   = driver.find_element(By.CLASS_NAME, "jobs-unified-top-card__job-title")
        job_comp_loc= driver.find_element(By.CLASS_NAME, "jobs-unified-top-card__primary-description")

        job_dict["Title"].append(job_title.text)
        job_dict["Company"].append(job_comp_loc.text.split(' · ')[0])
        job_dict["Location"].append(job_comp_loc.text.split(' · ')[1])
        job_dict["Description"].append(job_details.text)
        job_dict["URL"].append(driver.current_url)
        
        time.sleep(2)
    
#     job_details = driver.find_element(By.ID, "job-details")
#     print(job_details.text())
    


<selenium.webdriver.remote.webelement.WebElement (session="97665836d4f380cff48f79fa17a49649", element="22B30BE935252EAF494533384577A90E_element_2996")>
<selenium.webdriver.remote.webelement.WebElement (session="97665836d4f380cff48f79fa17a49649", element="22B30BE935252EAF494533384577A90E_element_2996")>


In [33]:
job_df = pd.DataFrame.from_dict(job_dict)
display(job_df)

job_df.to_excel("jobs_found.xlsx", index=False)

,Title,Company,Location,Description,URL
0,Staff Data Scientist,Peerlogic,"Scottsdale, AZ (On-site) 5 days ago","About the job\nLocation: Scottsdale, Arizona, ...",https://www.linkedin.com/jobs/search/?currentJ...
1,Data Engineer,Jobot,"Philadelphia, PA (On-site) 18 minutes ago",About the job\nWant to learn more about this r...,https://www.linkedin.com/jobs/search/?currentJ...
2,Data Scientist,Strategic Staffing Solutions,"Charlotte, NC (On-site) 3 weeks ago",About the job\nStrategic Staffing Solutions is...,https://www.linkedin.com/jobs/search/?currentJ...
3,Data Analyst,"DigiFlight, Inc.","Columbia, MD (On-site) Reposted 2 weeks ago",About the job\nWork with Senior Data Analysts ...,https://www.linkedin.com/jobs/search/?currentJ...
4,Data Analyst,Keller Postman LLC,"Orlando, FL (On-site) 5 days ago",About the job\nFirm Summary: Keller Postman is...,https://www.linkedin.com/jobs/search/?currentJ...
5,Data Analyst,"Staffing Resource Group, Inc","Arlington, VA (On-site) 2 weeks ago",About the job\nJob Description\n\nData Analyst...,https://www.linkedin.com/jobs/search/?currentJ...


In [28]:
job_title   = driver.find_element(By.CLASS_NAME, "jobs-unified-top-card__primary-description")
job_title.text.split(' · ')

['Staffing Resource Group, Inc',
 'Arlington, VA (On-site)  2 weeks ago ',
 '96 applicants']

In [59]:
driver.close()

## Extracting key words from resume